# Introcution to Fourier Transform 

## Definition and Basic Properties
The Fourier Transform is a mathematical tool that decomposes a time-domain signal into its constituent frequencies. It represents the signal in the frequency domain, where the amplitude and phase of each frequency component are expressed. The Fourier Transform of a continuous function $f(t)$ is defined as:

```{math}
:label: fourier_transform_definition
F(\omega) = \int_{-\infty}^{\infty} f(t) e^{-j\omega t} dt
```

Here:
- $F(\omega)$: Frequency-domain representation of $f(t)$.
- $\omega$: Angular frequency ($\omega = 2\pi f$).
- $t$: Time variable.
- $j$: Imaginary unit.

### Key Properties of Fourier Transform
1. **Linearity:** If $h(t) = a f(t) + b g(t)$, then $H(\omega) = a F(\omega) + b G(\omega)$.
2. **Time Shift:** A time shift in $f(t)$ results in a phase shift in $F(\omega)$:
   ```{math}
   :label: time_shift_property
   f(t - t_0) \xrightarrow{FT} e^{-j\omega t_0} F(\omega).
   ```
3. **Frequency Shift:** Modulating $f(t)$ by $e^{j\omega_0 t}$ shifts the spectrum:
   ```{math}
   :label: frequency_shift_property
   f(t)e^{j\omega_0 t} \xrightarrow{FT} F(\omega - \omega_0).
   ```
4. **Parseval's Theorem:** The energy of the signal in time domain equals the energy in frequency domain:
   ```{math}
   :label: parseval_theorem
   \int_{-\infty}^{\infty} |f(t)|^2 dt = \frac{1}{2\pi} \int_{-\infty}^{\infty} |F(\omega)|^2 d\omega.
   ```
More details on properties and applications of Fourier Transform can be found in the [Fourier Transform](https://en.wikipedia.org/wiki/Fourier_transform) Wikipedia page.

## Continuous vs. Discrete Fourier Transforms
While the continuous Fourier Transform applies to signals defined over an infinite time range, the Discrete Fourier Transform (DFT) is used for sampled signals. The DFT is defined as:

```{math}
:label: dft_definition
F[k] = \sum_{n=0}^{N-1} f[n] e^{-j \frac{2\pi}{N} kn}, \quad k = 0, 1, ..., N-1.
```

Here:
- $N$: Number of samples.
- $f[n]$: Sampled signal.
- $F[k]$: Discrete Fourier coefficients.

The DFT is computationally efficient when implemented using the Fast Fourier Transform (FFT), making it crucial in digital signal processing.

## Physical Interpretation in Acoustics
In acoustics, the Fourier Transform is vital for analyzing sound waves, which are time-domain signals, into their frequency components. This aids in understanding:
- **Harmonics:** Identifying frequencies present in a complex sound.
- **Resonances:** Peaks in the frequency spectrum corresponding to natural frequencies of a system.
- **Filters:** Designing systems to pass or block specific frequency components.

For instance, a complex waveform produced by musical instruments can be decomposed to reveal its fundamental frequency and overtones. Similarly, analyzing room acoustics often involves studying frequency-dependent reverberation.

## Interactive Demonstration I
Below is the Python code to run the interactive simulation, where you can
- Select the type of waveform (sine, square, triangular).
- Adjust the frequency slider to observe changes in both time and frequency domains.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

fmax =50

def plot_fourier(signal_type, freq):
    t = np.linspace(0, 1, 5000000)
    if signal_type == "Sine Wave":
        f = np.sin(2 * np.pi * freq * t)
    elif signal_type == "Square Wave":
        f = np.sign(np.sin(2 * np.pi * freq * t))
    elif signal_type == "Triangular Wave":
        f = 2 * np.abs(2 * ((t * freq) % 1) - 1) - 1

    N = len(t)
    F = np.fft.fft(f)
    F_magnitude = np.abs(F) / N
    F_single_sided = F_magnitude[: N // 2] * 2
    freq_axis = np.fft.fftfreq(N, d=(t[1]-t[0]))
    freq_single_sided = freq_axis[: N // 2]

    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.plot(t, f)
    plt.title(f"{signal_type} (Time Domain)")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")

    plt.subplot(2, 1, 2)
    plt.plot(freq_single_sided, F_single_sided)
    plt.title("Fourier Transform (Frequency Domain)")
    plt.axis([freq / 2, freq * 2, -0.1, 2]) # type: ignore
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude")
    plt.tight_layout()
    plt.show()

interact(plot_fourier, signal_type=["Sine Wave", "Square Wave", "Triangular Wave"], freq=(1, fmax, 1))

## Interactive Demonstration II
Below is the Python code to run the interactive simulation of transient response in a damped oscillatory system, where you can
- Understand Transient Response: Learn how how damping affects oscillations over time.
- Explore Fourier Transform & Spectrum Analysis: Explore how signals can be transformed into the frequency domain using FFT, identifying dominant frequency components.
- Adjust the frequency, damping factor, and duration to observe their impact on the transient response. slider to observe changes in both time and frequency domains.
- There is also an option to listen to the synthesized sound.

In [ ]:
import micropip
await micropip.install("IPython")
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import Audio


# Example usage
f = 10  # Frequency in Hz
delta = 3  # Damping factor
T = 3.0  # Duration in seconds
fs = 48000  # Sampling frequency in Hz
play_sound = 1  # 1 to play sound, 0 otherwise


def transient_response(f, delta, T, fs, ps):
    omega0 = 2 * np.pi * f
    dt = 1.0 / fs
    t = np.arange(0, T, dt)
    v0 = 1

    # Compute the transient response
    s = v0 * np.exp(-delta * t) * np.cos(omega0 * t)

    # Plot the time-domain response
    plt.figure(23)
    plt.plot(t, s, "k")
    plt.axis([0, T, -v0, v0])
    plt.xlabel("t (s)", fontsize=20)
    plt.ylabel("s (-)", fontsize=20)
    plt.gca().tick_params(labelsize=20)
    plt.grid(True)

    if ps == 1:
        audio_widget = Audio(s, rate=48000)
        display(audio_widget)

        print(" Play the sound.")

    return s


def plot_single_sided_spectrum(s, fs, f):

    # Perform FFT
    N = len(s)
    Y = fft(s)
    freq = fftfreq(N, 1 / fs)  # Frequency vector

    # Compute single-sided amplitude spectrum
    Y_magnitude = np.abs(Y) / N
    Y_single_sided = Y_magnitude[: N // 2] * 2
    freq_single_sided = freq[: N // 2]

    # Plot single-sided amplitude spectrum
    plt.figure()
    plt.plot(freq_single_sided, Y_single_sided, "k")
    plt.axis([f / 2, f * 2, -0.1, 1.1])
    plt.title("Single-Sided Amplitude Spectrum")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()


def update_spectrum(f, delta, T, fs):
    s = transient_response(f, delta, T, fs, play_sound)
    plot_single_sided_spectrum(s, fs, f)


f_widget = widgets.IntSlider(min=1, max=500, step=1, value=f, description="Frequency (Hz)")
delta_widget = widgets.FloatSlider(
    min=0.1, max=10, step=0.1, value=delta, description="Damping Factor"
)
T_widget = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=T, description="Duration (s)")
fs_widget = widgets.fixed(fs)

ui_plot = widgets.interactive_output(
    update_spectrum,
    {"f": f_widget, "delta": delta_widget, "T": T_widget, "fs": fs_widget},
)

display(f_widget, delta_widget, T_widget, ui_plot)

## References
- {cite:p}`jacobsen2013fundamentals`
- {cite:p}`kuttruff2007acoustics`